NoSQL datapipeline
Apache Airflow - Orchestrador e Agendador
OpenWeatherAPI - api de tempo
Python (extrair api)
Pyspark - Transformar, validar e carregar
MongoDB
Criaçao de Dashboard interativo

In [12]:
%cd /content/drive/MyDrive/github_projects/nosql-openweather

/content/drive/MyDrive/github_projects/nosql-openweather


In [25]:
!pip install python-dotenv

In [31]:
from mnt.python.coletor_weather import ColetorWeather

# Lista de cidades (ex: capitais)
cidades = [
    'Rio Branco,BR',           # Acre (AC)
    'Maceió,BR',               # Alagoas (AL)
    'Macapá,BR',               # Amapá (AP)
    'Manaus,BR',               # Amazonas (AM)
    'Salvador,BR',             # Bahia (BA)
    'Fortaleza,BR',            # Ceará (CE)
    'Brasília,BR',             # Distrito Federal (DF)
    'Vitória,BR',              # Espírito Santo (ES)
    'Goiânia,BR',              # Goiás (GO)
    'São Luís,BR',             # Maranhão (MA)
    'Cuiabá,BR',               # Mato Grosso (MT)
    'Campo Grande,BR',         # Mato Grosso do Sul (MS)
    'Belo Horizonte,BR',       # Minas Gerais (MG)
    'Belém,BR',                # Pará (PA)
    'João Pessoa,BR',          # Paraíba (PB)
    'Curitiba,BR',             # Paraná (PR)
    'Recife,BR',               # Pernambuco (PE)
    'Teresina,BR',             # Piauí (PI)
    'Rio de Janeiro,BR',       # Rio de Janeiro (RJ)
    'Natal,BR',                # Rio Grande do Norte (RN)
    'Porto Alegre,BR',         # Rio Grande do Sul (RS)
    'Porto Velho,BR',          # Rondônia (RO)
    'Boa Vista,BR',            # Roraima (RR)
    'Florianópolis,BR',        # Santa Catarina (SC)
    'São Paulo,BR',            # São Paulo (SP)
    'Aracaju,BR',              # Sergipe (SE)
    'Palmas,BR'                # Tocantins (TO)
]

# Instancia e coleta
extract = ColetorWeather(cidades)
dados = extract.coletar()

# Salvar dados
extract.salvar_jsons(dados)



[✓] Coletado: Rio Branco,BR
[✓] Coletado: Maceió,BR
[✓] Coletado: Macapá,BR
[✓] Coletado: Manaus,BR
[✓] Coletado: Salvador,BR
[✓] Coletado: Fortaleza,BR
[✓] Coletado: Brasília,BR
[✓] Coletado: Vitória,BR
[✓] Coletado: Goiânia,BR
[✓] Coletado: São Luís,BR
[✓] Coletado: Cuiabá,BR
[✓] Coletado: Campo Grande,BR
[✓] Coletado: Belo Horizonte,BR
[✓] Coletado: Belém,BR
[✓] Coletado: João Pessoa,BR
[✓] Coletado: Curitiba,BR
[✓] Coletado: Recife,BR
[✓] Coletado: Teresina,BR
[✓] Coletado: Rio de Janeiro,BR
[✓] Coletado: Natal,BR
[✓] Coletado: Porto Alegre,BR
[✓] Coletado: Porto Velho,BR
[✓] Coletado: Boa Vista,BR
[✓] Coletado: Florianópolis,BR
[✓] Coletado: São Paulo,BR
[✓] Coletado: Aracaju,BR
[✓] Coletado: Palmas,BR


In [32]:
with open('/content/drive/MyDrive/github_projects/nosql-openweather/data/clima_Aracaju_2025-05-24_12-46-33.json', 'r', encoding='utf-8') as f:
    dados = json.load(f)

df = pd.json_normalize(data)

display(df)

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.sea_level,main.grnd_level,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 803, 'main': 'Clouds', 'description': ...",stations,10000,1748088775,-10800,3451190,Rio de Janeiro,200,-43.2075,-22.9028,...,1024,1026,3.09,50,75,2,2098643,BR,1748078576,1748117806


In [33]:
!sudo apt install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (74.0 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 126102 files and directories currently instal

In [35]:
!tree -L 2

.
├── data
│   ├── clima_Aracaju_2025-05-24_12-46-33.json
│   ├── clima_Belém_2025-05-24_12-46-33.json
│   ├── clima_Belo_Horizonte_2025-05-24_12-46-33.json
│   ├── clima_Boa_Vista_2025-05-24_12-46-33.json
│   ├── clima_Brasília_2025-05-24_12-46-33.json
│   ├── clima_Campo_Grande_2025-05-24_12-46-33.json
│   ├── clima_Cuiabá_2025-05-24_12-46-33.json
│   ├── clima_Curitiba_2025-05-24_12-46-33.json
│   ├── clima_Fortaleza_2025-05-24_12-46-33.json
│   ├── clima_fpolis_2025-05-24_12-46-33.json
│   ├── clima_Goiânia_2025-05-24_12-46-33.json
│   ├── clima_João_Pessoa_2025-05-24_12-46-33.json
│   ├── clima_Macapá_2025-05-24_12-46-33.json
│   ├── clima_Maceió_2025-05-24_12-46-33.json
│   ├── clima_Manaus_2025-05-24_12-46-33.json
│   ├── clima_Natal_2025-05-24_12-46-33.json
│   ├── clima_Palmas_2025-05-24_12-46-33.json
│   ├── clima_Porto_Alegre_2025-05-24_12-46-33.json
│   ├── clima_Porto_Velho_2025-05-24_12-46-33.json
│   ├── clima_Recife_2025-05-24_12-46-33.json
│   ├── clima_Rio_Bran